In [62]:
import cv2
import xml.etree.ElementTree as ET
import sys
import os
import re
import os.path as osp
from PIL import Image

In [110]:
img_paths = [osp.join(root, f) for root, _, files in os.walk('/media/allen/mass/sogo/testing/JPEGImages/') 
                               for f in files if 'jpg' in f or 'png' in f]
anno_paths = [osp.join(root, f) for root, _, files in os.walk('/media/allen/mass/sogo/testing/Annotations/') 
                               for f in files if 'xml' in f]

In [111]:
bboxes = {}
for path in img_paths:
    filename = osp.basename(path)
    anno = osp.splitext(filename)[0] + ".xml"
    anno = osp.join('/media/allen/mass/sogo/testing/Annotations/', anno)
    tree = ET.parse(anno)
    root = tree.getroot()
    temp = []
    for x in root.findall('object'):
        bbox =x.find('bndbox')
        x1 = int(bbox.find('xmin').text)
        y1 = int(bbox.find('ymin').text)
        x2 = int(bbox.find('xmax').text)
        y2 = int(bbox.find('ymax').text)
        temp.append((x1, y1, x2, y2))
    bboxes[path] = temp

In [112]:
os.makedirs('/media/allen/mass/reid/sogo/bounding_box_test')

FileExistsError: [Errno 17] File exists: '/media/allen/mass/reid/sogo/bounding_box_test'

In [96]:
img_paths[0]
pattern = re.compile(r'(\w\d)_[\d]+-[\d]+-[\d]+_[\w]+.mp4_(\d+)_with')
pattern.search(img_paths[0]).groups()

('E1', '3750')

In [113]:
cams = set()
pattern = re.compile(r'(\w\d)_[\d]+-[\d]+-[\d]+_[\w]+.mp4_(\d+)_with')
for path in img_paths:
    cam, _ = pattern.search(path).groups()
    cams.add(cam)

In [114]:
cams = {cam:idx for idx, cam in enumerate(sorted(list(cams)), 1)}

In [115]:
cams

{'A1': 1,
 'B2': 2,
 'C3': 3,
 'D4': 4,
 'E1': 5,
 'F2': 6,
 'G3': 7,
 'H4': 8,
 'I5': 9}

In [116]:
pattern = re.compile(r'(\w\d)_[\d]+-[\d]+-[\d]+_[\w]+.mp4_(\d+)_with')
for path in bboxes.keys():
    img = cv2.imread(path)
    cam, timestamp = pattern.search(path).groups()
    ext = osp.splitext(path)[-1]
    for i, bbox in enumerate(bboxes[path], 1):
        fname = "9999_c{}s1_{}_{:02}{}".format(cams[cam], timestamp, i, ext)
        fname = osp.join('/media/allen/mass/reid/sogo/bounding_box_test', fname)
        x1, y1, x2, y2 = bbox
        img_crop = img[y1:y2, x1:x2, :]
        cv2.imwrite(fname, img_crop)

In [101]:
fname

'/media/allen/mass/reid/sogo/bounding_box_train/9999_cE1s1_3750_07.jpg'